In [1]:
# Instalando bibliotecas

!pip install surprise
!pip install dill


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\hamme\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\hamme\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Importando pacotes

from surprise import Reader, Dataset
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

### CARREGANDO DADOS

Base de dados do Goodreads no Kaggle: 

Link: https://www.kaggle.com/code/philippsp/book-recommender-collaborative-filtering-shiny/input

In [3]:
# Dataset com o detalhe dos Livros
livros = pd.read_csv('books.csv')
livros.head(5)

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [4]:
# Dataset com rating dos livros
ratings = pd.read_csv('ratings.csv')
ratings.head(2)

,book_id,user_id,rating
0,1,314,5
1,1,439,3


In [5]:
# Dataset com lista de tags utilizadas no site Goodreads
tags = pd.read_csv('tags.csv')
tags.head(5)

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [6]:
# Dataset com tags aplicadas em livros
book_tags = pd.read_csv('book_tags.csv')
book_tags.head(5)

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


### Preparação dos Dados de Conteúdo (Tags)

In [38]:
# Preparação dos Dados de Conteúdo (Tags)
print("Preparando dados para Filtragem Baseada em Conteúdo (CB)...")

# Juntando 'book_tags' com 'tags' para obter os nomes das tags
book_tags_nomes = book_tags.merge(tags, on='tag_id')

# Agrupando as tags por livro (goodreads_book_id) em uma única string
book_tag_counts = book_tags_nomes.groupby('goodreads_book_id')['tag_name'].apply(lambda x: ' '.join(x)).reset_index()
book_tag_counts.columns = ['book_id', 'all_tags']

# Unindo com o DataFrame principal de livros
livros_para_cb = livros.merge(book_tag_counts, on='book_id')
livros_para_cb = livros_para_cb[['book_id', 'original_title', 'all_tags']].copy()
livros_para_cb = livros_para_cb.fillna('')

print("DataFrame CB preparado (amostra):")
print(livros_para_cb.head(2))

Preparando dados para Filtragem Baseada em Conteúdo (CB)...
DataFrame CB preparado (amostra):
   book_id                            original_title  \
0  2767052                          The Hunger Games   
1        3  Harry Potter and the Philosopher's Stone   

                                            all_tags  
0  favorites currently-reading young-adult fictio...  
1  to-read favorites fantasy currently-reading yo...  


### Criação da Matriz de Similaridade de Conteúdo (TF-IDF e Cosseno)

In [39]:
# 2. Criação da Matriz de Similaridade de Conteúdo (TF-IDF e Cosseno)

# Criando a matriz TF-IDF com base nas tags de todos os livros
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(livros_para_cb['all_tags'])
print('Tamanho da Matriz TF-IDF (Livros x Tags):', tfidf_matrix.shape)

# Calculando a similaridade do cosseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('Tamanho da Matriz de Similaridade de Cosseno:', cosine_sim.shape)

# Mapeamentos para buscar dados rapidamente
bookid_to_index = pd.Series(livros_para_cb.index, index=livros_para_cb['book_id']).drop_duplicates()
index_to_bookid = pd.Series(livros_para_cb['book_id'], index=livros_para_cb.index).drop_duplicates()

print("Mapeamentos criados.")

Tamanho da Matriz TF-IDF (Livros x Tags): (10000, 16396)
Tamanho da Matriz de Similaridade de Cosseno: (10000, 10000)
Mapeamentos criados.


### EDA - Análise Exploratória

In [9]:
# Juntando as informações de livros e avaliações
livros_ratings = ratings.join(livros.set_index('book_id'), on='book_id')
livros_ratings.head(2)

,book_id,user_id,rating,id,best_book_id,work_id,books_count,isbn,isbn13,authors,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,314,5,27.0,1.0,41335427.0,275.0,439785960,9.780440e+12,"J.K. Rowling, Mary GrandPré",...,1678823.0,1785676.0,27520.0,7308.0,21516.0,136333.0,459028.0,1161491.0,https://images.gr-assets.com/books/1361039191m...,https://images.gr-assets.com/books/1361039191s...
1,1,439,3,27.0,1.0,41335427.0,275.0,439785960,9.780440e+12,"J.K. Rowling, Mary GrandPré",...,1678823.0,1785676.0,27520.0,7308.0,21516.0,136333.0,459028.0,1161491.0,https://images.gr-assets.com/books/1361039191m...,https://images.gr-assets.com/books/1361039191s...


In [10]:
print('Tamanho da Base de Filmes: ', livros_ratings.shape)

Tamanho da Base de Filmes:  (981756, 25)


In [11]:
livros_ratings.describe()

,book_id,user_id,rating,id,best_book_id,work_id,books_count,isbn13,original_publication_year,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
count,981756.000000,981756.000000,981756.000000,79701.000000,7.970100e+04,7.970100e+04,79701.000000,7.930300e+04,79601.000000,79701.000000,7.970100e+04,7.970100e+04,79701.000000,79701.000000,79701.000000,79701.000000,7.970100e+04,7.970100e+04
mean,4943.275636,25616.759933,3.856534,3336.508513,2.880303e+05,2.084769e+06,179.014291,9.768289e+12,1939.075916,3.940815,1.435863e+05,1.549325e+05,4429.072960,3774.000552,8277.743065,29418.200575,5.106888e+04,6.239363e+04
std,2873.207415,15228.338826,0.983941,2898.695872,2.632681e+06,3.411570e+06,359.766233,3.443687e+11,293.751475,0.251891,3.519843e+05,3.719871e+05,8057.594656,10360.958972,19326.666191,61325.679544,1.127500e+05,1.851993e+05
min,1.000000,1.000000,1.000000,2.000000,1.000000e+00,8.700000e+01,2.000000,7.678361e+10,-762.000000,2.800000,5.321000e+03,8.012000e+03,38.000000,39.000000,134.000000,911.000000,2.174000e+03,7.810000e+02
25%,2457.000000,12372.000000,3.000000,790.000000,2.744000e+03,8.063110e+05,40.000000,9.780193e+12,1964.000000,3.780000,1.772300e+04,1.917500e+04,813.000000,281.000000,1009.000000,4438.000000,6.888000e+03,6.103000e+03
50%,4921.000000,25077.000000,4.000000,2564.000000,5.128000e+03,1.385044e+06,75.000000,9.780385e+12,1994.000000,3.950000,3.761300e+04,4.211300e+04,1600.000000,763.000000,2271.000000,9481.000000,1.479600e+04,1.327700e+04
75%,7414.000000,38572.000000,5.000000,5343.000000,7.199000e+03,2.713437e+06,130.000000,9.780680e+12,2001.000000,4.120000,1.139150e+05,1.226530e+05,4204.000000,2185.000000,6355.000000,24897.000000,4.291200e+04,4.613100e+04
max,10000.000000,53424.000000,5.000000,10000.000000,3.358672e+07,4.133543e+07,3455.000000,9.788424e+12,2009.000000,4.770000,4.602479e+06,4.800065e+06,75867.000000,109383.000000,197621.000000,606158.000000,1.156318e+06,3.011543e+06


In [12]:
# Números dos datasets
print('Quantidade de livros Avaliados: ',
livros_ratings['book_id'].value_counts().shape[0])

print('Quantidade de Usuários Avaliando: ',
livros_ratings['user_id'].value_counts().shape[0])

print('Quantidade de Avaliações: ',
ratings.shape[0])

Quantidade de livros Avaliados:  10000
Quantidade de Usuários Avaliando:  53424
Quantidade de Avaliações:  981756


In [13]:
# Quantidade de Avaliações TOP5 Livros
livros_ratings['original_title'].value_counts().head()

original_title
Harry Potter and the Half-Blood Prince      100
Drowning Ruth                               100
What Looks Like Crazy on an Ordinary Day    100
While I Was Gone                            100
Mother of Pearl                             100
Name: count, dtype: int64

In [14]:
# Quantidade de Avaliações LOW5 Livros
livros_ratings['original_title'].value_counts().tail()

original_title
Ghostwritten                                                                                                            60
Loving What Is: Four Questions That Can Change Your Life                                                                60
Shopaholic Abroad                                                                                                       59
Jesus Freaks: DC Talk and The Voice of the Martyrs - Stories of Those Who Stood for Jesus, the Ultimate Jesus Freaks    59
Shadow of the Hegemon                                                                                                   57
Name: count, dtype: int64

In [15]:
# Quantidade de Avaliações TOP5 Usuários
livros_ratings['user_id'].value_counts().head()

user_id
30944    200
12874    200
12381    199
52036    199
28158    199
Name: count, dtype: int64

In [16]:
# Quantidade de Avaliações LOW5 Usuários
livros_ratings['user_id'].value_counts().tail()

user_id
10351    2
16592    2
24343    2
41314    2
27590    2
Name: count, dtype: int64

In [17]:
# Avaliações do usuário 30944
livros_ratings.query('user_id == 30944').head()

,book_id,user_id,rating,id,best_book_id,work_id,books_count,isbn,isbn13,authors,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
164,2,30944,5,21.0,2.0,2809203.0,307.0,439358078,9.780439e+12,"J.K. Rowling, Mary GrandPré",...,1735368.0,1840548.0,28685.0,9528.0,31577.0,180210.0,494427.0,1124806.0,https://images.gr-assets.com/books/1387141547m...,https://images.gr-assets.com/books/1387141547s...
364,4,30944,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,5,30944,4,18.0,5.0,2402163.0,376.0,043965548X,9.780440e+12,"J.K. Rowling, Mary GrandPré, Rufus Beck",...,1832823.0,1969375.0,36099.0,6716.0,20413.0,166129.0,509447.0,1266670.0,https://images.gr-assets.com/books/1499277281m...,https://images.gr-assets.com/books/1499277281s...
671,7,30944,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
762,8,30944,3,3275.0,8.0,21457576.0,11.0,439682584,9.780440e+12,"J.K. Rowling, Mary GrandPré",...,33220.0,33424.0,156.0,317.0,233.0,978.0,3686.0,28210.0,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


### MODELO

In [18]:
# Configuração para treinamento
reader = Reader(rating_scale=(0,5))
# Seleção das variáveis para o modelo
data = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader)
# Divisão dos dados de treino e teste
trainset, testset = train_test_split(data, test_size=.25, random_state=42)

In [19]:
data.df.head()

,user_id,book_id,rating
0,314,1,5
1,439,1,3
2,588,1,5
3,1169,1,4
4,1185,1,4


In [20]:
# Configurações das medidas de similaridade
sim_options = { 'name': 'pearson_baseline', 'user_based': True }

### Treinamento do Modelo de Filtragem Colaborativa

KNNBaseline: algoritmo básico que leva em consideração uma classificação de linha de base.

In [21]:
# Criação do modelo
knn = KNNBaseline(k=33, sim_options=sim_options)
# Treinamento do modelo
knn.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


### Avaliação do Modelo

In [22]:
predictions_knn = knn.test(testset)
accuracy.rmse(predictions_knn)

RMSE: 0.8831


0.883105710918292

### Predição do Modelo

Parâmetros que são atribuidos as predições.

Parameters:

uid – The (raw) user id. See this note.     
iid – The (raw) item id. See this note.     
r_ui (float) – The true rating rui.     
est (float) – The estimated rating r^ui.     
details (dict) – Stores additional details about the prediction that might be useful for later analysis.

In [23]:
# Predição com os dados de teste
predictions_knn[:5]

[Prediction(uid=28306, iid=9302, r_ui=5.0, est=3.9169017244319053, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=18736, iid=834, r_ui=3.0, est=3.3209277500584915, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid=22386, iid=2338, r_ui=3.0, est=3.9817988601238508, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid=6441, iid=5278, r_ui=5.0, est=3.2486558887954824, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=28576, iid=2378, r_ui=4.0, est=4.237133431619651, details={'actual_k': 7, 'was_impossible': False})]

A função trainset utiliza um id interno para serem utilizados no modelo,     
eles podem ser obtidos através da função to_inner, e a função to_raw     
retorna os valores originais.

In [24]:
# uid=247, iid=364 Conversão para o id interno
print('Conversão Interna')
print('User:', trainset.to_inner_uid(274))
print('Livro:', trainset.to_inner_iid(1))

# uid=247, iid=364 Conversão para o id externo
print('Conversão Externa')
print('User:', trainset.to_raw_uid( trainset.to_inner_uid(274) ))
print('Livro:', trainset.to_raw_iid( trainset.to_inner_iid(1) ))

Conversão Interna
User: 4188
Livro: 4974
Conversão Externa
User: 274
Livro: 1


### Função Análise de Recomendação Colaborativa

Passando um ID de usuário e um ID de um livro, a função vai retornar a     
Estimativa de Avaliação do usuário para o livro informado. Caso seja um     
livro já avaliado pelo usuário retorna também a avaliação real.

In [49]:
def recomenda_livro(user_id,book_id):
  # ID do usuário para predição
  uid = user_id
  # ID do filme para predição
  iid = book_id
  nome_livro = livros.query('book_id == @book_id')['original_title'].values[0]
  print('Livro:', nome_livro)
  print('Usuário:', user_id)
  if livros_ratings.query('user_id == @user_id and book_id == @book_id')['original_title'].values.size == 0:
    print('Usuário não avaliou o livro!')
  else:
    nota_livro = ratings.query('user_id == @user_id and book_id == @book_id')['rating'].values[0]
    print('Avaliação do usuário:', nota_livro)
  # Predição baseada no melhor modelo
  print('Estimativa de Avaliação[0-5]:', round(knn.predict(trainset.to_raw_uid(uid), trainset.to_raw_iid(iid))[3], 2))

In [83]:
# Estimativa de Avaliação do usuário para o livro informado
recomenda_livro(280,1)

Livro: Harry Potter and the Half-Blood Prince
Usuário: 280
Usuário não avaliou o livro!
Estimativa de Avaliação[0-5]: 3.76


In [84]:
# Estimativa de Avaliação do usuário para o livro informado
recomenda_livro(998,1)

Livro: Harry Potter and the Half-Blood Prince
Usuário: 998
Usuário não avaliou o livro!
Estimativa de Avaliação[0-5]: 3.59


### Função TOP Recomendações

Passando um ID de usuário e a quantidade de recomendações, a função informa     
em ordem de recomendação os filmes para o usuário informado.

In [29]:
def top_n(user_id,n):
  # Selecionando apenas os livros do treinamento
  lista_livros_treino = []
  for x in trainset.all_items():
    lista_livros_treino.append(trainset.to_raw_iid(x))
  # Selecionando os livros do treinamento que o usuário não avaliou
  livros_ratings_user = ratings.query('user_id == @user_id')['book_id'].values
  livros_ratings_user_nao = livros.query('book_id not in @livros_ratings_user')
  livros_ratings_user_nao = livros_ratings_user_nao.query('book_id in @lista_livros_treino')['book_id'].values
  # Criando um ranking para o usuário para os livros não avaliados
  ranking=[]
  for book_id in livros_ratings_user_nao:
    ranking.append((book_id, knn.predict(trainset.to_inner_uid(user_id), trainset.to_inner_iid(book_id))[3]))
  # Ordenando os TOP livros avaliados
  ranking.sort(key=lambda x: x[1], reverse=True)
  # Selecionando os Ids dos livros
  x,_ = zip(*ranking[:n])
  # Listando os nomes dos livros em ordem de recomendação
  return livros.query('book_id in @x')['original_title'].copy().reset_index(drop=True)

In [30]:
class Recomenda:
    def __init__(self, ratings, livros, trainset):
        self.ratings = ratings
        self.movies = livros
        self.trainset = trainset

    def top_n(self,user_id,n):
      # Selecionando apenas os livros do treinamento
      lista_livros_treino = []
      for x in trainset.all_items():
        lista_livros_treino.append(trainset.to_raw_iid(x))
      # Selecionando os livros do treinamento que o usuário não avaliou
      livros_ratings_user = ratings.query('user_id == @user_id')['book_id'].values
      livros_ratings_user_nao = livros.query('book_id not in @livros_ratings_user')
      livros_ratings_user_nao = livros_ratings_user_nao.query('book_id in @lista_livros_treino')['book_id'].values
      # Criando um ranking para o usuário para os livros não avaliados
      ranking=[]
      for book_id in livros_ratings_user_nao:
        ranking.append((book_id, knn.predict(trainset.to_inner_uid(user_id), trainset.to_inner_iid(book_id))[3]))
      # Ordenando os TOP livros avaliados
      ranking.sort(key=lambda x: x[1], reverse=True)
      # Selecionando os Ids dos livros
      x,_ = zip(*ranking[:n])
      # Listando os nomes dos livros em ordem de recomendação
      return livros.query('book_id in @x')['original_title'].copy().reset_index(drop=True)

In [31]:
class Recomenda:
    def __init__(self, livros_ratings):
        self.livros_ratings = livros_ratings

    def top_n(self,n):
      # Quantidade de Avaliações TOP5 Usuários
      return self.livros_ratings['user_id'].value_counts().head(n)

In [ ]:
# Apresentado é a lista dos 10 usuários que fizeram o maior número de avaliações no dataset.
recomenda = Recomenda(livros_ratings)
print(recomenda.top_n(10))

user_id
30944    200
12874    200
12381    199
52036    199
28158    199
45554    197
37834    197
6630     197
7563     196
9668     196
Name: count, dtype: int64


In [33]:
import sys
# Instala o pacote dill no interpretador que o notebook está usando
!{sys.executable} -m pip install dill

In [34]:
import dill

with open('meu_objeto.pkl', 'wb') as file:
    dill.dump(recomenda, file)

In [72]:
# Com base na Filtragem Colaborativa, recomenda 5 livros para o usuário 280
top_n(280,5)

0                The Great Gatsby
1                The Time Machine
2    Memoria de mis putas tristes
3                          Truman
4              Il barone rampante
Name: original_title, dtype: object

In [ ]:
# Com base na Filtragem Colaborativa, recomenda 5 livros para o usuário 998
top_n(998,5)

0                    BLEACH―ブリーチ―　1
1                 The Mermaid Chair
2     Haroun and the Sea of Stories
3       The Man from St. Petersburg
4    A Home at the End of the World
Name: original_title, dtype: object

### Função Híbrida por Cascata

Filtragem Híbrida:     
Aplicando Filtragem Colaborativa para selecionar 2 livros iniciais e, em seguida,     
utilizar Filtragem de Conteúdo para buscar outros 3 livros semelhantes aos 2 livros iniciais.

In [85]:
# FUNÇÃO TOP N HÍBRIDA POR CASCATA (SEQUENCIAL) 

def top_n_hibrida_cascata(user_id, n_cf=2, n_cb=3, model=knn):
    # (Códigos das linhas 1 a 18 da função original)

    # 1.1. Encontrar livros não avaliados 
    rated_books_ids = set(ratings.query('user_id == @user_id')['book_id'].values)
    all_books_ids = set(livros['book_id'].values)
    unrated_books_ids = list(all_books_ids - rated_books_ids)
    
    # 1.2. Criar dataset de teste apenas com IDs que o modelo conhece 
    unrated_books_test_set = []
    
    for book_id in unrated_books_ids:
        # A. Verifica se o livro existe na base de Conteúdo (CB)
        if book_id not in bookid_to_index.index:
            continue
            
        # B. Verifica se o livro (raw ID) é conhecido pelo modelo KNN (CF)
        # O método .knows_item() requer o ID interno, o que é problemático.
        # A maneira mais direta e correta é verificar se o raw ID está no dicionário de mapeamento
        # interno-para-bruto (ou vice-versa), que é mais robusto:
        if book_id in model.trainset._raw2inner_id_items:
             unrated_books_test_set.append((user_id, book_id, 4)) # 4 é rating dummy
             
    # 1.3. Fazer predições e selecionar o Top n_cf
    predictions = model.test(unrated_books_test_set)
    predictions.sort(key=lambda x: x.est, reverse=True)
    top_cf_predictions = predictions[:n_cf]
    
    cf_book_ids = [pred.iid for pred in top_cf_predictions]
    cf_titles = livros.query('book_id in @cf_book_ids')['original_title'].values.tolist()
    
    # 2. ETAPA CB: Recomendação Baseada em Conteúdo (Top n_cb)
      
    # 2.1. Lista de IDs a serem excluídos do resultado final do CB
    excluded_ids = set(cf_book_ids) | rated_books_ids
    
    # 2.2. Encontrar os índices na matriz de similaridade para os n_cf livros (Seeds)
    seed_indices = [bookid_to_index[b_id] for b_id in cf_book_ids if b_id in bookid_to_index.index]
    
    if not seed_indices:
        print("Aviso: Os livros recomendados por CF não foram encontrados na base de conteúdo. Retornando apenas CF.")
        return pd.Series(cf_titles)

    cb_scores = {}
    
    # 2.3. Iterar sobre todos os livros (candidatos) para calcular o score CB
    for i in livros_para_cb.index:
        candidate_book_id = index_to_bookid[i]
        
        # Ignorar livros já recomendados pelo CF ou já avaliados
        if candidate_book_id in excluded_ids:
            continue
            
        # Calcula a similaridade máxima com os livros "seed" (o max é uma heurística comum para o CB)
        max_similarity = np.max(cosine_sim[i, seed_indices])
        
        # Armazena o score CB (similaridade)
        cb_scores[candidate_book_id] = max_similarity

    # 2.4. Selecionar o Top n_cb
    sorted_cb_scores = sorted(cb_scores.items(), key=lambda item: item[1], reverse=True)
    top_cb_book_ids = [item[0] for item in sorted_cb_scores[:n_cb]]
    
    cb_titles = livros.query('book_id in @top_cb_book_ids')['original_title'].values.tolist()
    
    # 3. RESULTADO FINAL (UNIÃO)
    final_titles = cf_titles + cb_titles
    
    return pd.Series(final_titles)

### Execução do Modelo Híbrido em Cascata

In [86]:
# --- Execução do Modelo Híbrido em Cascata (2 CF + 3 CB) ---
print("Top 5 Recomendações Híbridas em Cascata para o Usuário 280:")
recs_cascata_280 = top_n_hibrida_cascata(user_id=280, n_cf=2, n_cb=3)
print(recs_cascata_280)

print("\nTop 5 Recomendações Híbridas em Cascata para o Usuário 998:")
recs_cascata_998 = top_n_hibrida_cascata(user_id=998, n_cf=2, n_cb=3)
print(recs_cascata_998)

Top 5 Recomendações Híbridas em Cascata para o Usuário 280:
0                    Still Life with Woodpecker
1                                 O Monte Cinco
2    Na margem do rio Piedra eu sentei e chorei
3                                       O Zahir
4                                         Brida
dtype: object

Top 5 Recomendações Híbridas em Cascata para o Usuário 998:
0     Still Life with Woodpecker
1                Villa Incognito
2              Jitterbug Perfume
3            Skinny Legs and All
4    Half Asleep in Frog Pajamas
dtype: object
